In [3]:
import pandas as pd
import os
from markdown import markdown

In [4]:
df = pd.read_csv("suspects_5MSV.txt", sep="\t")

In [5]:
df.head()

,SuspectIndex,DeltaMZ,Cosine,PutativeID,Compound_Name,Adduct,Precursor_MZ,INCHI,SpectrumID,LibraryQualityString,#Scan#,MZErrorPPM,SharedPeaks,Suspect,cluster index,ScanNumber,Original_Path,cluster index_library,ScanNumber_library,Original_Path_library
0,172,-14.0200,0.818500,CH2,C17-Sphinganine,M+H,288.291,,CCMSLIB00000579284,Bronze,147,3.38742,7,Suspect related to C17-Sphinganine - CH2,172.0,34.0,f.MSV000078547/spectrum/DLab/121207_proximicin...,147.0,302.0,f.MSV000078547/spectrum/DLab/fi1026d_FT-it.mzXML
1,1551,-15.9980,0.835552,O,Beauvericin,M+Na,806.399,InChI=1S/C45H57N3O9/c1-28(2)37-40(49)46(7)35(2...,CCMSLIB00003416348,Bronze,1520,1.66515,7,Suspect related to Beauvericin - O,1551.0,139.0,f.MSV000078547/spectrum/DLab/121207_aby_J_DD_M...,1520.0,29.0,f.MSV000078547/spectrum/DLab/121207_aby_J_DD_M...
2,386,-15.9900,0.831178,O,Tyr-Pro-Phe,M+H,426.202,InChI=1S/C23H27N3O5/c24-18(13-16-8-10-17(27)11...,CCMSLIB00003729791,Bronze,410,18.04410,6,Suspect related to Tyr-Pro-Phe - O,386.0,132.0,f.MSV000078547/spectrum/DLab/fi1026d_FT-it.mzXML,410.0,97.0,f.MSV000078547/spectrum/DLab/fi1026c_FT-it.mzXML
3,193,-28.0308,0.970810,C2H4,Pseudopyronine B,M+H,295.228,,CCMSLIB00000223956,Bronze,238,9.61337,6,Suspect related to Pseudopyronine B - C2H4,193.0,162.0,f.MSV000078552/spectrum/Pseudomonas/Pseudomona...,238.0,530.0,f.MSV000078552/spectrum/Pseudomonas/Pseudomona...
4,138096,-28.0360,0.945772,C2H4,Massbank:FIO00053 delta-Dodecalactone,[M+H]+,199.170,1S/C12H22O2/c1-2-3-4-5-6-8-11-9-7-10-12(13)14-...,CCMSLIB00000213680,Bronze,43603,4.97977,7,Suspect related to Massbank:FIO00053 delta-Dod...,138096.0,2599.0,f.MSV000078556/spectrum/Human_Swabs/Woman/13m_...,43603.0,2270.0,f.MSV000078556/spectrum/Human_Swabs/Woman/m15_...


In [10]:
df["dataset"] = df["Original_Path"].apply(lambda x: x[2:14])
df["usi1"] = df.apply(lambda x: "mzspec:{}:{}:scan:{}".format(x["dataset"], os.path.basename(x["Original_Path"]), x["ScanNumber"]) , axis=1)
df["usi2"] = df.apply(lambda x: "mzspec:GNPSLIBRARY:{}".format(x["SpectrumID"]) , axis=1)
df["usi3"] = df.apply(lambda x: "mzspec:MSV000084314:{}.mgf:scan:{}".format(x["dataset"], x["#Scan#"]) , axis=1)

In [13]:
# Writing Markdown File

output_list = []
output_list.append("Suspect | Mirror Library | Mirror Dataset Cluster |Image")
output_list.append("--- | --- | --- | ---")
for record in df.to_dict(orient="records"):
    spectrum_url = "https://metabolomics-usi.ucsd.edu/svg/?usi={}&mz_min=50&mz_max=500".format(record["usi1"])
    mirror_url = "https://metabolomics-usi.ucsd.edu/svg/mirror?usi1={}&usi2={}&mz_min=50&mz_max=500&max_intensity=150".format(record["usi1"], record["usi2"])
    mirror_alt_url = "https://metabolomics-usi.ucsd.edu/svg/mirror?usi1={}&usi2={}&mz_min=50&mz_max=500&max_intensity=150".format(record["usi1"], record["usi3"])
    output_list.append("{} | ![]({}) | ![]({}) | ![]({})".format(record["Suspect"], mirror_url, mirror_alt_url, spectrum_url))

In [14]:
print("\n".join(output_list))

with open("suspect_library_20200501.md", "w") as outfile:
    outfile.write("\n".join(output_list))

with open("suspect_library_20200501.html", "w") as outfile:
    outfile.write(markdown("\n".join(output_list), extensions=['tables']))

an:2895.0&usi2=mzspec:GNPSLIBRARY:CCMSLIB00000075360&mz_min=50&mz_max=500&max_intensity=150) | ![](https://metabolomics-usi.ucsd.edu/svg/mirror?usi1=mzspec:MSV000078556:12m__BC5_01_1949.mzXML:scan:2895.0&usi2=mzspec:MSV000084314:MSV000078556.mgf:scan:413086&mz_min=50&mz_max=500&max_intensity=150) | ![](https://metabolomics-usi.ucsd.edu/svg/?usi=mzspec:MSV000078556:12m__BC5_01_1949.mzXML:scan:2895.0&mz_min=50&mz_max=500)
Suspect related to Octadecenoic acid - C2H4 | ![](https://metabolomics-usi.ucsd.edu/svg/mirror?usi1=mzspec:MSV000078556:m15__BA7_01_2236.mzXML:scan:2886.0&usi2=mzspec:GNPSLIBRARY:CCMSLIB00000075360&mz_min=50&mz_max=500&max_intensity=150) | ![](https://metabolomics-usi.ucsd.edu/svg/mirror?usi1=mzspec:MSV000078556:m15__BA7_01_2236.mzXML:scan:2886.0&usi2=mzspec:MSV000084314:MSV000078556.mgf:scan:413086&mz_min=50&mz_max=500&max_intensity=150) | ![](https://metabolomics-usi.ucsd.edu/svg/?usi=mzspec:MSV000078556:m15__BA7_01_2236.mzXML:scan:2886.0&mz_min=50&mz_max=500)
Suspect